# Import and Device

In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# If fdi_models.py is in the same directory as the notebook:
from fdi_models import BiLSTMFDIDetector, count_parameters

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

Using device: cuda


# Load Prepared Dataset

In [27]:
DATA_PATH = "prepared_data/smartgrid_fdi_seq32.npz"  

data = np.load(DATA_PATH, allow_pickle=True)
print("Keys:", data.files)

X_train = data["X_train"]
y_train = data["y_train"]
X_val   = data["X_val"]
y_val   = data["y_val"]
X_test  = data["X_test"]
y_test  = data["y_test"]

print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val:  ", X_val.shape,   "y_val:  ", y_val.shape)
print("X_test: ", X_test.shape,  "y_test: ", y_test.shape)

print("Train label counts:", np.bincount(y_train.astype(int)))
print("Val label counts:  ", np.bincount(y_val.astype(int)))
print("Test label counts: ", np.bincount(y_test.astype(int)))



Keys: ['X_train', 'y_train', 'X_val', 'y_val', 'X_test', 'y_test']
X_train: (1249, 32, 15) y_train: (1249,)
X_val:   (156, 32, 15) y_val:   (156,)
X_test:  (157, 32, 15) y_test:  (157,)
Train label counts: [624 625]
Val label counts:   [78 78]
Test label counts:  [79 78]


# Organizing Dataset to Dataloaders

In [28]:
class SequenceFDIDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).float()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

BATCH_SIZE = 128

train_ds = SequenceFDIDataset(X_train, y_train)
val_ds   = SequenceFDIDataset(X_val,   y_val)
test_ds  = SequenceFDIDataset(X_test,  y_test)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False)

len(train_ds), len(val_ds), len(test_ds)




(1249, 156, 157)

# Instantiate BiLSTM from .py

In [38]:
input_dim = X_train.shape[2]

model = BiLSTMFDIDetector(
    input_dim=input_dim,
    hidden_dim=64,
    num_layers=1,
    bidirectional=True,
    dropout=0.25,
).to(DEVICE)

print(model)
print("Trainable parameters:", count_parameters(model))

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

BiLSTMFDIDetector(
  (lstm): LSTM(15, 64, batch_first=True, bidirectional=True)
  (classifier): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Linear(in_features=64, out_features=1, bias=True)
  )
)
Trainable parameters: 49793


# Training Loop

In [39]:

def batch_accuracy_from_logits(logits: torch.Tensor, labels: torch.Tensor) -> float:
    probs = torch.sigmoid(logits)
    preds = (probs >= 0.5).float()
    return (preds == labels).float().mean().item()

def evaluate(model, data_loader, criterion):
    model.eval()
    total_loss = 0.0
    total_acc = 0.0
    total_n = 0

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.to(DEVICE)
            y_batch = y_batch.to(DEVICE)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            n = y_batch.size(0)
            total_loss += loss.item() * n
            total_acc  += batch_accuracy_from_logits(logits, y_batch) * n
            total_n += n

    return total_loss / total_n, total_acc / total_n


NUM_EPOCHS = 100
best_val_loss = float("inf")

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    total_n = 0

    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        n = y_batch.size(0)
        running_loss += loss.item() * n
        running_acc  += batch_accuracy_from_logits(logits, y_batch) * n
        total_n += n

    train_loss = running_loss / total_n
    train_acc  = running_acc / total_n
    val_loss, val_acc = evaluate(model, val_loader, criterion)

    print(
        f"Epoch {epoch:02d} | "
        f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
        f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}"
    )

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_bilstm_fdi_detector.pt")
        print("  -> Saved new best model.")





Epoch 01 | Train Loss: 0.6935, Acc: 0.4988 | Val Loss: 0.6931, Acc: 0.5000
  -> Saved new best model.
Epoch 02 | Train Loss: 0.6927, Acc: 0.5148 | Val Loss: 0.6931, Acc: 0.5064
Epoch 03 | Train Loss: 0.6929, Acc: 0.5172 | Val Loss: 0.6931, Acc: 0.5385
  -> Saved new best model.
Epoch 04 | Train Loss: 0.6935, Acc: 0.4924 | Val Loss: 0.6931, Acc: 0.4872
Epoch 05 | Train Loss: 0.6924, Acc: 0.5260 | Val Loss: 0.6930, Acc: 0.5192
  -> Saved new best model.
Epoch 06 | Train Loss: 0.6922, Acc: 0.5268 | Val Loss: 0.6930, Acc: 0.5256
  -> Saved new best model.
Epoch 07 | Train Loss: 0.6920, Acc: 0.5180 | Val Loss: 0.6927, Acc: 0.5000
  -> Saved new best model.
Epoch 08 | Train Loss: 0.6919, Acc: 0.5444 | Val Loss: 0.6927, Acc: 0.5000
  -> Saved new best model.
Epoch 09 | Train Loss: 0.6917, Acc: 0.5236 | Val Loss: 0.6925, Acc: 0.5192
  -> Saved new best model.
Epoch 10 | Train Loss: 0.6914, Acc: 0.5292 | Val Loss: 0.6922, Acc: 0.5064
  -> Saved new best model.
Epoch 11 | Train Loss: 0.6906, Acc

# Test Evaluation

In [40]:
# Load best model for testing
model.load_state_dict(torch.load("best_bilstm_fdi_detector.pt", map_location=DEVICE))
model.to(DEVICE)
model.eval()

all_logits = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)

        logits = model(X_batch)
        all_logits.append(logits.cpu().numpy())
        all_labels.append(y_batch.cpu().numpy())

all_logits = np.concatenate(all_logits)
all_labels = np.concatenate(all_labels)

probs = 1 / (1 + np.exp(-all_logits))   # sigmoid
preds = (probs >= 0.5).astype(np.float32)

acc  = accuracy_score(all_labels, preds)
prec = precision_score(all_labels, preds)
rec  = recall_score(all_labels, preds)
f1   = f1_score(all_labels, preds)
cm   = confusion_matrix(all_labels, preds)

print("Test Accuracy :", acc)
print("Test Precision:", prec)
print("Test Recall   :", rec)
print("Test F1       :", f1)
print("Confusion matrix:\n", cm)

tn, fp, fn, tp = cm.ravel()
print(f"TN={tn}, FP={fp}, FN={fn}, TP={tp}")

Test Accuracy : 0.5222929936305732
Test Precision: 0.5272727272727272
Test Recall   : 0.3717948717948718
Test F1       : 0.43609022556390975
Confusion matrix:
 [[53 26]
 [49 29]]
TN=53, FP=26, FN=49, TP=29


/tmp/ipykernel_20645/4031893533.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_bilstm_fdi_detector.pt", map_location=DEVICE))
